In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG

%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [3]:
DATA_DIR = CONFIG.CFG.DATA.BASE

In [4]:
def convertTrainLabel():
    df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
    bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        df[column] = bboxs[:,i]
    df.drop(columns=['bbox'], inplace=True)
    df['x_center'] = df['x'] + df['w']/2
    df['y_center'] = df['y'] + df['h']/2
    df['classes'] = 0
    from tqdm.auto import tqdm
    import shutil as sh
    df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
    
    index = list(set(df.image_id))
    
    source = 'train'
    if True:
        for fold in [0]:
            val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
            for name,mini in tqdm(df.groupby('image_id')):
                if name in val_index:
                    path2save = 'val2017/'
                else:
                    path2save = 'train2017/'
                if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                    os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
                with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                    row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                    row = row/1024
                    row = row.astype(str)
                    for j in range(len(row)):
                        text = ' '.join(row[j])
                        f.write(text)
                        f.write("\n")
                if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                    os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
                sh.copy("{}/{}/{}.jpg".format(DATA_DIR, source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [6]:
convertTrainLabel()

In [7]:
CONFIG.upload_to_kaggle("yolov5model", "YOLOv5X MODEL Weights")

In [ ]:
#git clone https://github.com/NVIDIA/apex && cd apex && python3 -m pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex
##!git clone https://github.com/ultralytics/yolov5
## python3 -m pip install -r requirements.txt

!python3 train.py --img 1024 --batch 2 --epochs 35 --data ../Kaggle-Global-Wheat-Detection/YOLOv5/wheat0.yaml --cfg ../Kaggle-Global-Wheat-Detection/YOLOv5/yolov5x.yaml --name yolov5x_fold0 --weights